In [ ]:
# ==============================================================================
# Bloco 1: Importação das bibliotecas e carregamento dos dados
# ==============================================================================
import pandas as pd
import numpy as np


# Para exibir os gráficos no próprio notebook
%matplotlib inline 
import matplotlib.pyplot as plt
import seaborn as sns

print("Bibliotecas importadas com sucesso!")

# Carregando o arquivo principal com os resultados dos jogos
# O arquivo está na pasta 'data' que fica um nível acima da pasta 'notebooks'
try:
    df_full = pd.read_csv('../data/campeonato-brasileiro-full.csv')
    print("Arquivo 'campeonato-brasileiro-full.csv' carregado com sucesso!")
    print(f"O dataset contém {df_full.shape[0]} partidas e {df_full.shape[1]} colunas.")
except FileNotFoundError:
    print("ERRO: O arquivo 'campeonato-brasileiro-full.csv' não foi encontrado.")
    print("Por favor, certifique-se de que ele está na pasta 'data' no diretório raiz do projeto.")

# Exibindo as primeiras linhas para entender a estrutura
if 'df_full' in locals():
    df_full.head()

In [ ]:
# ==============================================================================
# Carregamento dos arquivos adicionais da pasta data
# ==============================================================================
# Carregando os outros arquivos disponíveis para análises mais detalhadas

print("Carregando arquivos adicionais...")

try:
    # Arquivo com dados de cartões
    df_cartoes = pd.read_csv('../data/campeonato-brasileiro-cartoes.csv')
    print(f"✓ Cartões: {df_cartoes.shape[0]} registros, {df_cartoes.shape[1]} colunas")
except FileNotFoundError:
    print("✗ Arquivo de cartões não encontrado")
    df_cartoes = None

try:
    # Arquivo com estatísticas completas
    df_estatisticas = pd.read_csv('../data/campeonato-brasileiro-estatisticas-full.csv')
    print(f"✓ Estatísticas: {df_estatisticas.shape[0]} registros, {df_estatisticas.shape[1]} colunas")
except FileNotFoundError:
    print("✗ Arquivo de estatísticas não encontrado")
    df_estatisticas = None

try:
    # Arquivo com dados de gols
    df_gols = pd.read_csv('../data/campeonato-brasileiro-gols.csv')
    print(f"✓ Gols: {df_gols.shape[0]} registros, {df_gols.shape[1]} colunas")
except FileNotFoundError:
    print("✗ Arquivo de gols não encontrado")
    df_gols = None

print("\nTodos os arquivos de dados foram processados!")

In [ ]:
# ==============================================================================
# Bloco 2: Limpeza e Preparação dos Dados
# ==============================================================================
# Vamos trabalhar com as colunas de placar. Elas podem não ser numéricas.
# Convertemos para número, e qualquer erro (ex: placar não preenchido) vira 'NaN' (Not a Number)
if 'df_full' in locals():
	df_full['mandante_Placar'] = pd.to_numeric(df_full['mandante_Placar'], errors='coerce')
	df_full['visitante_Placar'] = pd.to_numeric(df_full['visitante_Placar'], errors='coerce')

	# Removemos as linhas onde o placar não é um número (jogos que talvez não aconteceram ou dados faltantes)
	df_jogos_realizados = df_full.dropna(subset=['mandante_Placar', 'visitante_Placar']).copy()

	# Convertemos os placares para inteiros, pois agora sabemos que são todos números
	df_jogos_realizados['mandante_Placar'] = df_jogos_realizados['mandante_Placar'].astype(int)
	df_jogos_realizados['visitante_Placar'] = df_jogos_realizados['visitante_Placar'].astype(int)

	print("Limpeza dos dados de placar concluída.")
	print(f"Analisando um total de {len(df_jogos_realizados)} partidas com resultados válidos.")
else:
	print("ERRO: O dataframe 'df_full' não está definido. Certifique-se de que o arquivo foi carregado corretamente.")

In [ ]:
# ==============================================================================
# Bloco 3: Cálculo das Métricas Gerais do Campeonato
# ==============================================================================

if 'df_jogos_realizados' in locals():
	# 1. Total de Gols Marcados
	df_jogos_realizados['total_gols'] = df_jogos_realizados['mandante_Placar'] + df_jogos_realizados['visitante_Placar']
	total_gols = df_jogos_realizados['total_gols'].sum()

	# 2. Total de Partidas
	total_partidas = len(df_jogos_realizados)

	# 3. Média de Gols por Partida
	media_gols_partida = total_gols / total_partidas

	# 4. Contagem de Vitórias (Mandante, Visitante) e Empates
	vitorias_mandante = df_jogos_realizados[df_jogos_realizados['vencedor'] == df_jogos_realizados['mandante']].shape[0]
	vitorias_visitante = df_jogos_realizados[df_jogos_realizados['vencedor'] == df_jogos_realizados['visitante']].shape[0]
	empates = df_jogos_realizados[df_jogos_realizados['vencedor'] == '-'].shape[0]

	# 5. Calculando os Percentuais
	perc_vitoria_mandante = (vitorias_mandante / total_partidas) * 100
	perc_vitoria_visitante = (vitorias_visitante / total_partidas) * 100
	perc_empates = (empates / total_partidas) * 100

	# Exibindo os resultados de forma organizada
	print("--- Análise Geral do Campeonato (Dados de 2003 a 2024) ---")
	print("="*55)
	print(f"Total de Partidas Analisadas: {total_partidas}")
	print(f"Total de Gols Marcados: {total_gols}")
	print(f"Média de Gols por Partida: {media_gols_partida:.2f}")
	print("-"*55)
	print("Distribuição dos Resultados:")
	print(f"  - Vitórias do Mandante: {vitorias_mandante} ({perc_vitoria_mandante:.2f}%)")
	print(f"  - Vitórias do Visitante: {vitorias_visitante} ({perc_vitoria_visitante:.2f}%)")
	print(f"  - Empates: {empates} ({perc_empates:.2f}%)")
	print("="*55)
else:
	print("ERRO: O dataframe 'df_jogos_realizados' não está definido. Certifique-se de que o arquivo foi carregado e os dados foram preparados corretamente.")

In [ ]:
# ==============================================================================
# Bloco 4: Visualização Gráfica da Distribuição dos Resultados
# ==============================================================================
# Criando um gráfico de pizza para visualizar os percentuais

# Dados para o gráfico
labels = ['Vitórias do Mandante', 'Vitórias do Visitante', 'Empates']
sizes = [vitorias_mandante, vitorias_visitante, empates]
cores = ['#4CAF50', '#2196F3', '#FFC107'] # Verde, Azul, Amarelo
explode = (0.05, 0, 0)  # "Explodir" a primeira fatia para dar destaque

# Criando a figura e o eixo
fig1, ax1 = plt.subplots(figsize=(8, 8))
ax1.pie(sizes, explode=explode, labels=labels, colors=cores, autopct='%1.1f%%',
        shadow=True, startangle=90, textprops={'fontsize': 14})
ax1.axis('equal')  # Garante que o gráfico seja um círculo.

# Adicionando um título
plt.title('Distribuição Percentual dos Resultados\n(Todas as Temporadas)', fontsize=16)

# Mostrando o gráfico
plt.show()

In [ ]:
# ==============================================================================
# Bloco 5: Cálculo do Desempenho Agregado por Equipe
# ==============================================================================
print("Iniciando a análise de desempenho por equipe...")

# Pegamos uma lista com o nome de todos os times únicos do dataset
times = pd.unique(df_jogos_realizados[['mandante', 'visitante']].values.ravel('K'))

# Lista para armazenar as estatísticas de cada time
stats_times = []

# Loop para calcular as estatísticas de cada time
for time in times:
    # Jogos como mandante
    jogos_mandante = df_jogos_realizados[df_jogos_realizados['mandante'] == time]
    # Jogos como visitante
    jogos_visitante = df_jogos_realizados[df_jogos_realizados['visitante'] == time]
    
    # Total de jogos
    total_jogos = len(jogos_mandante) + len(jogos_visitante)
    
    # Vitórias
    vitorias = (jogos_mandante['vencedor'] == time).sum() + (jogos_visitante['vencedor'] == time).sum()
    
    # Empates
    empates = (jogos_mandante['vencedor'] == '-').sum() + (jogos_visitante['vencedor'] == '-').sum()
    
    # Derrotas
    derrotas = total_jogos - vitorias - empates
    
    # Gols Marcados (Gols Pró)
    gols_pro = jogos_mandante['mandante_Placar'].sum() + jogos_visitante['visitante_Placar'].sum()
    
    # Gols Sofridos (Gols Contra)
    gols_contra = jogos_mandante['visitante_Placar'].sum() + jogos_visitante['mandante_Placar'].sum()
    
    # Saldo de Gols
    saldo_gols = gols_pro - gols_contra
    
    # Pontos
    pontos = (vitorias * 3) + empates
    
    # Adicionando o dicionário de stats à nossa lista
    stats_times.append({
        'Clube': time,
        'Pts': pontos,
        'J': total_jogos,
        'V': vitorias,
        'E': empates,
        'D': derrotas,
        'GP': gols_pro,
        'GC': gols_contra,
        'SG': saldo_gols
    })

# Criando um DataFrame com as estatísticas
df_performance = pd.DataFrame(stats_times)

# Ordenando o DataFrame pelos pontos (e depois saldo de gols e gols pró como critério de desempate)
df_performance = df_performance.sort_values(by=['Pts', 'SG', 'GP'], ascending=False).reset_index(drop=True)

print("\nTabela de desempenho agregado de todos os times (2003-2024) calculada com sucesso!")

# Exibindo os 10 times com melhor desempenho histórico
df_performance.head(10)

In [ ]:
# ==============================================================================
# Bloco 6: Análise de Força - Desempenho como Mandante vs. Visitante
# ==============================================================================
print("Iniciando a análise de força ofensiva e defensiva (Casa vs. Fora)...")

# Lista para armazenar as estatísticas detalhadas
stats_detalhadas = []

# Usamos a lista de times que já tínhamos
for time in times:
    # --- Desempenho em CASA ---
    jogos_casa = df_jogos_realizados[df_jogos_realizados['mandante'] == time]
    gols_pro_casa = jogos_casa['mandante_Placar'].sum()
    gols_contra_casa = jogos_casa['visitante_Placar'].sum()
    num_jogos_casa = len(jogos_casa)
    
    # Média de Gols Marcados em Casa (evita divisão por zero se o time nunca jogou em casa)
    media_gp_casa = gols_pro_casa / num_jogos_casa if num_jogos_casa > 0 else 0
    # Média de Gols Sofridos em Casa
    media_gc_casa = gols_contra_casa / num_jogos_casa if num_jogos_casa > 0 else 0

    # --- Desempenho FORA ---
    jogos_fora = df_jogos_realizados[df_jogos_realizados['visitante'] == time]
    gols_pro_fora = jogos_fora['visitante_Placar'].sum()
    gols_contra_fora = jogos_fora['mandante_Placar'].sum()
    num_jogos_fora = len(jogos_fora)

    # Média de Gols Marcados Fora
    media_gp_fora = gols_pro_fora / num_jogos_fora if num_jogos_fora > 0 else 0
    # Média de Gols Sofridos Fora
    media_gc_fora = gols_contra_fora / num_jogos_fora if num_jogos_fora > 0 else 0
    
    stats_detalhadas.append({
        'Clube': time,
        'Média Gols Marcados (Casa)': media_gp_casa,
        'Média Gols Sofridos (Casa)': media_gc_casa,
        'Média Gols Marcados (Fora)': media_gp_fora,
        'Média Gols Sofridos (Fora)': media_gc_fora
    })

# Criando o DataFrame de Força
df_forca = pd.DataFrame(stats_detalhadas)

print("\nTabela de força ofensiva e defensiva calculada com sucesso!")

# Exibindo os 10 times com melhor média de gols marcados em casa
df_forca.sort_values(by='Média Gols Marcados (Casa)', ascending=False).head(10)

In [ ]:
# ==============================================================================
# Bloco 8: Criação da Variável Alvo (Target) para o Modelo
# ==============================================================================
print("Criando a variável alvo 'Over_2_5'...")

# A condição `df_jogos_realizados['total_gols'] > 2.5` retorna True ou False.
# O comando `.astype(int)` converte True para 1 e False para 0.
df_jogos_realizados['Over_2_5'] = (df_jogos_realizados['total_gols'] > 2.5).astype(int)

print("Variável alvo criada com sucesso!")

# Vamos verificar como ficou a distribuição entre jogos com mais e menos de 2.5 gols
distribuicao_over_under = df_jogos_realizados['Over_2_5'].value_counts(normalize=True) * 100

print("\nDistribuição dos Resultados:")
print(f"  - Partidas com Menos de 2.5 gols (0): {distribuicao_over_under[0]:.2f}%")
print(f"  - Partidas com Mais de 2.5 gols (1): {distribuicao_over_under[1]:.2f}%")


# Exibindo as últimas colunas do DataFrame para vermos o resultado
df_jogos_realizados[['mandante', 'visitante', 'mandante_Placar', 'visitante_Placar', 'total_gols', 'Over_2_5']].head()

In [ ]:
# ==============================================================================
# Bloco 9: Visualização da Distribuição da Variável Alvo
# ==============================================================================
# Um gráfico de barras é ótimo para ver o balanço das nossas classes

# Criando a figura
plt.figure(figsize=(8, 6))

# Criando o gráfico com o Seaborn
ax = sns.countplot(x='Over_2_5', data=df_jogos_realizados, palette=['#FFC107', '#4CAF50'])

# Adicionando título e rótulos
plt.title('Distribuição de Partidas: Over/Under 2.5 Gols', fontsize=16)
plt.ylabel('Quantidade de Partidas', fontsize=12)
plt.xlabel('Resultado', fontsize=12)
ax.set_xticklabels(['Menos de 2.5 Gols', 'Mais de 2.5 Gols']) # Customizando os nomes no eixo X

# Adicionando os valores no topo das barras
for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 9), textcoords='offset points', fontsize=12)


plt.show()

In [ ]:
# ==============================================================================
# Bloco 10: Unindo Features e Target para Criar o Dataset do Modelo
# ==============================================================================
print("Iniciando a montagem do dataset final...")

# Primeiro, vamos preparar nosso dataframe de força para a junção
# Definimos o 'Clube' como índice para facilitar a busca de dados
df_forca_map = df_forca.set_index('Clube')

# --- Features do Mandante ---
# Usamos .map() para buscar a Média de Gols Marcados (Casa) de cada time mandante
df_jogos_realizados['mandante_media_GP_casa'] = df_jogos_realizados['mandante'].map(df_forca_map['Média Gols Marcados (Casa)'])
# E a Média de Gols Sofridos (Casa)
df_jogos_realizados['mandante_media_GC_casa'] = df_jogos_realizados['mandante'].map(df_forca_map['Média Gols Sofridos (Casa)'])

# --- Features do Visitante ---
# Usamos .map() para buscar a Média de Gols Marcados (Fora) de cada time visitante
df_jogos_realizados['visitante_media_GP_fora'] = df_jogos_realizados['visitante'].map(df_forca_map['Média Gols Marcados (Fora)'])
# E a Média de Gols Sofridos (Fora)
df_jogos_realizados['visitante_media_GC_fora'] = df_jogos_realizados['visitante'].map(df_forca_map['Média Gols Sofridos (Fora)'])

print("Métricas de força adicionadas a cada partida com sucesso!")

# --- Criação do DataFrame final do Modelo ---
# Selecionamos apenas as colunas que nosso modelo irá usar: as 4 features e o nosso alvo.
colunas_modelo = [
    'mandante_media_GP_casa',
    'mandante_media_GC_casa',
    'visitante_media_GP_fora',
    'visitante_media_GC_fora',
    'Over_2_5' # Nosso alvo!
]

df_modelo = df_jogos_realizados[colunas_modelo].copy()

# Removemos qualquer linha que possa ter ficado com dados faltantes (se algum time não tiver stats)
df_modelo.dropna(inplace=True)

print("\nDataFrame final para o treinamento do modelo foi criado!")
print(f"Ele contém {df_modelo.shape[0]} jogos e {df_modelo.shape[1]} colunas.")

# Exibindo as primeiras linhas do nosso dataset final pronto para o modelo
df_modelo.head()

In [ ]:
# ==============================================================================
# Bloco 11: Preparação, Treinamento e Avaliação do Modelo
# ==============================================================================
# Bibliotecas de Machine Learning do Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Bibliotecas de Machine Learning importadas com sucesso!")

# 1. Separando as Features (X) e o Alvo (y)
X = df_modelo.drop('Over_2_5', axis=1) # X são todas as colunas, exceto o nosso alvo
y = df_modelo['Over_2_5']             # y é apenas a coluna alvo

print(f"\nNossas features (X): {list(X.columns)}")
print(f"Nosso alvo (y): {y.name}")

# 2. Dividindo os dados em conjuntos de Treino e Teste (80% para treino, 20% para teste)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, # Proporção dos dados para o conjunto de teste
    random_state=42 # Garante que a divisão seja a mesma toda vez que rodarmos o código
)

print(f"\nDados divididos: {len(X_train)} amostras para treino e {len(X_test)} para teste.")

# 3. Criando e Treinando o Modelo Naive Bayes Gaussiano
# "Gaussiano" é a variante do Naive Bayes ideal para features com valores contínuos (como nossas médias de gols)
modelo_nb = GaussianNB()

print("\nTreinando o modelo Naive Bayes...")
# O método .fit() é o momento em que o modelo "aprende" com os dados de treino
modelo_nb.fit(X_train, y_train)
print(">>> Modelo treinado com sucesso!")

# 4. Fazendo Previsões com os Dados de Teste
print("\nFazendo previsões no conjunto de teste...")
previsoes = modelo_nb.predict(X_test)

# 5. Avaliando a Performance do Modelo
print("\n--- Avaliação do Modelo ---")
print("="*55)

# Acurácia: a métrica mais simples. Percentual de acertos.
acuracia = accuracy_score(y_test, previsoes)
print(f"Acurácia do Modelo: {acuracia * 100:.2f}%")
print("-"*55)

# Relatório de Classificação: mostra métricas mais detalhadas como precisão e recall
print("Relatório de Classificação:")
# Classe 0 = Under 2.5 gols | Classe 1 = Over 2.5 gols
print(classification_report(y_test, previsoes, target_names=['Under 2.5', 'Over 2.5']))
print("-"*55)

# Matriz de Confusão: uma tabela que mostra os acertos e erros em detalhe
print("Matriz de Confusão:")
cm = confusion_matrix(y_test, previsoes)
print("Explicação da Matriz:")
print(f"  - Previu 'Under' e acertou: {cm[0][0]}")
print(f"  - Previu 'Over' e errou (era 'Under'): {cm[0][1]}")
print(f"  - Previu 'Under' e errou (era 'Over'): {cm[1][0]}")
print(f"  - Previu 'Over' e acertou: {cm[1][1]}")

In [ ]:
# ==============================================================================
# Bloco 12: Visualização da Matriz de Confusão
# ==============================================================================
# Uma visualização gráfica da matriz de confusão é muito mais fácil de interpretar

# Criando a figura
plt.figure(figsize=(8, 6))

# Criando o heatmap com o Seaborn
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Previsto: Under 2.5', 'Previsto: Over 2.5'],
            yticklabels=['Real: Under 2.5', 'Real: Over 2.5'])

# Adicionando título e rótulos
plt.title('Matriz de Confusão', fontsize=16)
plt.ylabel('Resultado Real', fontsize=12)
plt.xlabel('Resultado Previsto pelo Modelo', fontsize=12)

plt.show()